In [1]:
path = "/home/krzysztof/Dokumenty/badania_iitis/tensors_symetric/tensor calculations/pictures_tensor/low_rank_tensor_approx/low-rank-tensor-approximation/src/"
include(path*"cumulants.jl")
include(path*"read_pictures.jl")
include(path*"moment_calc.jl")
include(path*"random.jl")

generate_random_gauss (generic function with 1 method)

In [3]:
rmprocs(workers())
addprocs()

8-element Array{Int64,1}:
 10
 11
 12
 13
 14
 15
 16
 17

In [54]:
data, n = generate_random_gauss(5, 1000000);

In [7]:
data, n = read_hyperspectral("indian_pines_hyperspectral.npy", 1);
data = data/maximum(data);

In [8]:
C2 = get_cumulant2(data);
r, X2, S = cov_calc(C2, n);

In [9]:
@time C3 = get_cumulant3(convert(Array{Float32,2},data));
X3 = hosvd(C3).matrices[1];

179.438818 seconds (27.21 k allocations: 82.140 MB, 0.02% gc time)


In [ ]:
@time C4 = get_cumulant4(convert(Array{Float32,2},data));
X4 = hosvd(C4).matrices[1];

In [68]:
using MAT
matwrite("cumulants.mat", Dict{Any,Any}(
    "C2" => C2,
    "C3" => C3,
    "C4" => C4,
    "X2" => X2,
    "X3" => X3,
    "X4" => X4
))

In [7]:
using MAT
matwrite("cumulants.mat", Dict{Any,Any}(
    "C2" => C2,
    "C3" => C3
))

In [70]:
norm_tensor(C3)

0.00045140120997063223

In [25]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "")
@pyimport cross_cumulant as cc
data1 = data - repmat(mean(data,1), size(data,1))

In [26]:
cc.compute_cumulant2(data1')-C2

In [74]:
cc.compute_cumulant3(data1')-C3

3x3 Array{Float64,2}:
 0.431195  0.328736  0.180378
 0.328736  0.254933  0.139698
 0.180378  0.139698  0.077555

In [75]:
cc.compute_cumulant4(data1')-C4

3x3 Array{Float64,2}:
  0.00612226   0.000111728  -6.69157e-6 
  0.000111728  0.000873043   0.000242232
 -6.69157e-6   0.000242232   0.00100628 